In [1]:
import os
import time
from datetime import datetime, timezone

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from aipricepatterns import Client

pd.set_option('display.max_columns', 80)
pd.set_option('display.width', 160)

def safe_float(x, default=0.0) -> float:
    try:
        return float(x)
    except Exception:
        return float(default)

def map_suggested_action_to_pos(x) -> int:
    """Map suggestedAction to position: -1/0/+1."""
    if x is None:
        return 0
    if isinstance(x, (int, float)):
        v = int(x)
        if v in (-1, 0, 1):
            return int(v)
        if v in (0, 1, 2):
            return 1 if v == 1 else (-1 if v == 2 else 0)
        return 0
    if not isinstance(x, str):
        return 0
    s = x.strip().lower()
    if s in ('hold', 'flat', 'none', 'neutral', 'wait'):
        return 0
    if s in ('long', 'buy', 'bull', 'up'):
        return 1
    if s in ('short', 'sell', 'bear', 'down'):
        return -1
    return 0

def policy_episode_pnl(ep: dict, horizon: int, trade_cost_pct: float, policy: str, conf_thr: float) -> float:
    ts = ep.get('transitions')
    if not isinstance(ts, list) or not ts:
        return 0.0
    sim = safe_float(ep.get('similarity'), 0.0)
    steps = min(len(ts), int(horizon))
    pos = 0
    pnl = 0.0
    cost = 0.0
    for i in range(steps):
        t = ts[i] if isinstance(ts[i], dict) else {}
        ret = safe_float(t.get('ret', t.get('return', 0.0)), 0.0)
        old_pos = pos

        if policy == 'alwaysFlat':
            pos = 0
        elif policy == 'alwaysLong':
            pos = 1
        elif policy == 'alwaysShort':
            pos = -1
        elif policy == 'suggested':
            pos = map_suggested_action_to_pos(t.get('suggestedAction'))
        elif policy == 'suggestedIfConf':
            pos = map_suggested_action_to_pos(t.get('suggestedAction')) if sim >= conf_thr else 0
        else:
            pos = 0

        if pos != old_pos and trade_cost_pct != 0.0:
            cost += abs(pos - old_pos) * (trade_cost_pct / 100.0)

        pnl += float(pos) * ret

    return float(pnl - cost)

def best_threshold_for_anchor(df_ep: pd.DataFrame, thresholds: np.ndarray) -> dict:
    # policy: if similarity < thr -> flat, else follow suggested (episode-level filter)
    rows = []
    n = len(df_ep)
    if n == 0:
        return {'bestThr': None, 'bestExpectedOverall': None, 'bestCoverage': None}
    for thr in thresholds:
        sub = df_ep[df_ep['similarity'] >= thr]
        if len(sub) == 0:
            continue
        coverage = float(len(sub) / n)
        avg_active = float(sub['pnl_suggested'].mean())
        expected_overall = float(avg_active * coverage)
        rows.append({'thr': float(thr), 'coverage': coverage, 'avgActive': avg_active, 'expectedOverall': expected_overall})
    if not rows:
        return {'bestThr': None, 'bestExpectedOverall': None, 'bestCoverage': None}
    best = max(rows, key=lambda r: r['expectedOverall'])
    return {'bestThr': float(best['thr']), 'bestExpectedOverall': float(best['expectedOverall']), 'bestCoverage': float(best['coverage'])}

## Parameters
We sweep **300 anchors** on BTCUSDT 1h by default. Tune knobs via env vars to avoid heavy API usage.

In [2]:
BASE_URL = os.getenv('AIPP_BASE_URL', 'https://aipricepatterns.com/api/rust')
API_KEY = os.getenv('AIPP_API_KEY')

SYMBOL = os.getenv('AIPP_RL_SYMBOL', 'BTCUSDT')
INTERVAL = os.getenv('AIPP_RL_INTERVAL', '1h')

ANCHOR_POINTS = int(os.getenv('AIPP_SWEEP_ANCHORS', '300'))
LOOKBACK_DAYS = int(os.getenv('AIPP_SWEEP_LOOKBACK_DAYS', '120'))

FORECAST_HORIZON = int(os.getenv('AIPP_RL_HORIZON', '24'))
EPISODES_PER_ANCHOR = int(os.getenv('AIPP_SWEEP_EPISODES_PER_ANCHOR', '200'))
MIN_SIMILARITY = float(os.getenv('AIPP_RL_MIN_SIMILARITY', '0.70'))
SAMPLING_STRATEGY = os.getenv('AIPP_RL_SAMPLING_STRATEGY', 'uniform')

TRADE_COST_PCT = float(os.getenv('AIPP_RL_TRADE_COST_PCT', '0.00'))
SUGGESTED_MIN_SIM = float(os.getenv('AIPP_RL_SUGGESTED_MIN_SIMILARITY', '0.90'))

THR_GRID_N = int(os.getenv('AIPP_SWEEP_THR_GRID_N', '21'))
SWEEP_SLEEP_SEC = float(os.getenv('AIPP_SWEEP_SLEEP_SEC', '0.10'))

CACHE_DIR = os.getenv('AIPP_RESEARCH_CACHE_DIR', 'python-sdk/research/_cache')
CACHE_PATH = os.path.join(CACHE_DIR, f'03_anchor_sweep_{SYMBOL}_{INTERVAL}_{ANCHOR_POINTS}.csv')

print('Base URL:', BASE_URL)
print(f'Symbol: {SYMBOL}  Interval: {INTERVAL}')
print(f'Anchors: {ANCHOR_POINTS}  LookbackDays: {LOOKBACK_DAYS}')
print(f'Episodes/anchor: {EPISODES_PER_ANCHOR}  minSimilarity: {MIN_SIMILARITY:.2f}  horizon: {FORECAST_HORIZON}')
print(f'TradeCostPct: {TRADE_COST_PCT:.4f}%  ConfThr: {SUGGESTED_MIN_SIM:.2f}')
print('Cache:', CACHE_PATH)

Base URL: https://aipricepatterns.com/api/rust
Symbol: BTCUSDT  Interval: 1h
Anchors: 300  LookbackDays: 120
Episodes/anchor: 200  minSimilarity: 0.70  horizon: 24
TradeCostPct: 0.0000%  ConfThr: 0.90
Cache: python-sdk/research/_cache/03_anchor_sweep_BTCUSDT_1h_300.csv


## Generate anchors
We place `ANCHOR_POINTS` timestamps uniformly over the last `LOOKBACK_DAYS`.

In [3]:
now_ms = int(time.time() * 1000)
start_ms = now_ms - LOOKBACK_DAYS * 24 * 60 * 60 * 1000
anchors = np.linspace(start_ms, now_ms, num=ANCHOR_POINTS, dtype=np.int64).tolist()

print('anchor range:', datetime.fromtimestamp(anchors[0]/1000, tz=timezone.utc), '->', datetime.fromtimestamp(anchors[-1]/1000, tz=timezone.utc))
print('example anchors:', [datetime.fromtimestamp(a/1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M') for a in anchors[:3]], '...')

anchor range: 2025-08-20 21:46:39.552000+00:00 -> 2025-12-18 21:46:39.552000+00:00
example anchors: ['2025-08-20 21:46', '2025-08-21 07:24', '2025-08-21 17:02'] ...


## Sweep (with caching)
If the cache exists, we load it. Otherwise we query the API and write a CSV cache.

In [4]:
os.makedirs(CACHE_DIR, exist_ok=True)

if os.path.exists(CACHE_PATH):
    sweep = pd.read_csv(CACHE_PATH)
    print('loaded cache:', CACHE_PATH, 'rows:', len(sweep))
else:
    client = Client(base_url=BASE_URL, api_key=API_KEY)

    rows = []
    for idx, anchor_ts in enumerate(anchors, start=1):
        res = client.get_rl_episodes(
            symbol=SYMBOL,
            interval=INTERVAL,
            anchor_ts=int(anchor_ts),
            forecast_horizon=FORECAST_HORIZON,
            num_episodes=EPISODES_PER_ANCHOR,
            min_similarity=MIN_SIMILARITY,
            include_actions=True,
            reward_type='returns',
            sampling_strategy=SAMPLING_STRATEGY,
        )
        eps = res.get('episodes') if isinstance(res, dict) else None
        if not isinstance(eps, list) or not eps:
            # keep going; write a placeholder row
            rows.append({
                'anchorTs': int(anchor_ts),
                'anchorDtUtc': datetime.fromtimestamp(anchor_ts/1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M'),
                'episodes': 0,
                'meanSimilarity': np.nan,
                'bestThr': np.nan,
                'bestExpectedOverall': np.nan,
                'bestCoverage': np.nan,
                'avgSuggested': np.nan,
                'avgSuggestedIfConf': np.nan,
            })
            continue

        sim = np.array([safe_float(ep.get('similarity'), np.nan) for ep in eps], dtype=float)
        df_ep = pd.DataFrame({'similarity': sim})

        df_ep['pnl_suggested'] = [policy_episode_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, 'suggested', SUGGESTED_MIN_SIM) for ep in eps]
        df_ep['pnl_suggestedIfConf'] = [policy_episode_pnl(ep, FORECAST_HORIZON, TRADE_COST_PCT, 'suggestedIfConf', SUGGESTED_MIN_SIM) for ep in eps]

        thr_min = float(np.nanmin(sim))
        thr_max = float(np.nanmax(sim))
        thr_grid = np.linspace(thr_min, thr_max, num=THR_GRID_N)
        best = best_threshold_for_anchor(df_ep, thr_grid)

        rows.append({
            'anchorTs': int(anchor_ts),
            'anchorDtUtc': datetime.fromtimestamp(anchor_ts/1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M'),
            'episodes': int(len(eps)),
            'meanSimilarity': float(np.nanmean(sim)),
            'bestThr': best['bestThr'],
            'bestExpectedOverall': best['bestExpectedOverall'],
            'bestCoverage': best['bestCoverage'],
            'avgSuggested': float(df_ep['pnl_suggested'].mean()),
            'avgSuggestedIfConf': float(df_ep['pnl_suggestedIfConf'].mean()),
        })

        if idx % 10 == 0:
            print(f'{idx}/{len(anchors)} anchors...')
        time.sleep(SWEEP_SLEEP_SEC)

    sweep = pd.DataFrame(rows)
    sweep.to_csv(CACHE_PATH, index=False)
    print('wrote cache:', CACHE_PATH, 'rows:', len(sweep))

sweep.head()

10/300 anchors...
20/300 anchors...
30/300 anchors...
40/300 anchors...
50/300 anchors...
60/300 anchors...
70/300 anchors...
80/300 anchors...
90/300 anchors...
100/300 anchors...
110/300 anchors...
120/300 anchors...
130/300 anchors...
140/300 anchors...
150/300 anchors...
160/300 anchors...
170/300 anchors...
180/300 anchors...
190/300 anchors...
200/300 anchors...
210/300 anchors...
220/300 anchors...
230/300 anchors...
240/300 anchors...
250/300 anchors...
260/300 anchors...
270/300 anchors...
280/300 anchors...
290/300 anchors...
300/300 anchors...
wrote cache: python-sdk/research/_cache/03_anchor_sweep_BTCUSDT_1h_300.csv rows: 300


,anchorTs,anchorDtUtc,episodes,meanSimilarity,bestThr,bestExpectedOverall,bestCoverage,avgSuggested,avgSuggestedIfConf
0,1755726399552,2025-08-20 21:46,200,0.933511,0.9216,5.065077,1.0,5.065077,5.065077
1,1755761075137,2025-08-21 07:24,200,0.893444,0.8847,5.935948,1.0,5.935948,0.771060
2,1755795750722,2025-08-21 17:02,200,0.889334,0.8755,9.016179,1.0,9.016179,1.419632
3,1755830426307,2025-08-22 02:40,200,0.936398,0.9283,10.206564,1.0,10.206564,10.206564
4,1755865101893,2025-08-22 12:18,200,0.953588,0.9471,8.429334,1.0,8.429334,8.429334


## Aggregate view
We look at stability and drift: how the best threshold and expectedOverall move over time.

In [5]:
sweep_clean = sweep.dropna(subset=['bestThr', 'bestExpectedOverall', 'bestCoverage', 'meanSimilarity']).copy()
print('valid anchors:', len(sweep_clean), '/', len(sweep))

sweep_clean[['meanSimilarity', 'bestThr', 'bestCoverage', 'bestExpectedOverall', 'avgSuggested', 'avgSuggestedIfConf']].describe().T

valid anchors: 300 / 300


,count,mean,std,min,25%,50%,75%,max
meanSimilarity,300.0,0.921742,0.049145,0.725093,0.895896,0.933293,0.961786,0.983671
bestThr,300.0,0.910521,0.053984,0.706100,0.880150,0.921200,0.954600,0.979700
bestCoverage,300.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
bestExpectedOverall,300.0,7.243477,2.065753,3.902701,5.711734,6.760547,8.434110,15.099652
avgSuggested,300.0,7.243477,2.065753,3.902701,5.711734,6.760547,8.434110,15.099652
avgSuggestedIfConf,300.0,5.459474,3.639895,0.000000,2.046660,5.923596,7.863574,15.099652


In [6]:
fig = px.histogram(sweep_clean, x='bestThr', nbins=30, title='Distribution of best similarity threshold (walk-forward)')
fig.update_layout(height=380)
fig

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sweep_clean['anchorTs'], y=sweep_clean['bestThr'], mode='lines', name='bestThr'))
fig.add_trace(go.Scatter(x=sweep_clean['anchorTs'], y=sweep_clean['meanSimilarity'], mode='lines', name='meanSimilarity', yaxis='y2'))
fig.update_layout(
    title='Threshold drift vs mean similarity',
    xaxis_title='anchorTs (ms)',
    yaxis=dict(title='bestThr'),
    yaxis2=dict(title='meanSimilarity', overlaying='y', side='right'),
    height=420,
)
fig

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sweep_clean['anchorTs'], y=sweep_clean['bestExpectedOverall'], mode='lines', name='bestExpectedOverall'))
fig.add_trace(go.Scatter(x=sweep_clean['anchorTs'], y=sweep_clean['bestCoverage'], mode='lines', name='bestCoverage', yaxis='y2'))
fig.update_layout(
    title='ExpectedOverall vs coverage over time',
    xaxis_title='anchorTs (ms)',
    yaxis=dict(title='expectedOverall (avgActive * coverage)'),
    yaxis2=dict(title='coverage', overlaying='y', side='right'),
    height=420,
)
fig

## Interpretation checklist
- If the best threshold distribution is tight, gating is stable.
- If it drifts widely, similarity may be non-stationary; consider regime conditioning or recalibration.
- If coverage collapses for high thresholds, your retrieval is overconfident / clustered.